In [1]:
from torchvision import datasets
from torchvision import transforms
import groundingdino.datasets.transforms as T
import os
from torch.utils.data import DataLoader
from lang_sam import LangSAM
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset
import json
import time
import gc
import lang_sam.utils as utils
from PIL import Image
from segment_anything.utils.transforms import ResizeLongestSide

/home/amin/miniconda3/envs/lsa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!gdown 12ni2gB-qTQIUp2H8Fgc5gzINW0o2Zw6z
!gdown 1AfA7r9sZVnM3cOCAK1AQTUN0d5PWkrRP
#!gdown 1v54AI5kqrGpe4Y2zXuxwz4sy9YOrPMk_
#!unzip -q Coco_Subset.zip

Downloading...
From: https://drive.google.com/uc?id=12ni2gB-qTQIUp2H8Fgc5gzINW0o2Zw6z
To: /home/amin/LangSam/new_objects.csv
100%|██████████████████████████████████████| 77.2M/77.2M [00:25<00:00, 2.97MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AfA7r9sZVnM3cOCAK1AQTUN0d5PWkrRP
To: /home/amin/LangSam/result.json
100%|██████████████████████████████████████| 55.2M/55.2M [00:18<00:00, 2.98MB/s]


In [5]:
model = LangSAM()


final text_encoder_type: bert-base-uncased


Model loaded from /home/amin/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [6]:
H,W=600,800

In [7]:
root="train2017"
files=sorted(os.listdir(root))
with open("result.json","r") as f:
    json_dict=json.load(f)

In [11]:
from tqdm import tqdm

In [26]:
new_dict={}
files_count=len(files)
i=10
subset=10000
begin=i*subset
end=begin+subset
for index in tqdm(range(begin,end)):
    image_pil=Image.open(root+"/"+files[index])
    image_pil=image_pil.resize((800,600))
    if image_pil.mode != "RGB":
        continue
    key=int(files[index].split(".")[0])
    if str(key) in json_dict.keys():
        new_dict[key]={}
        new_dict[key]["caption"]=json_dict[str(key)]["caption"]
        new_dict[key]["objects"]=json_dict[str(key)]["objects"]
        new_dict[key]["found_objects"]={}
        for object in json_dict[str(key)]["found_objects"].keys():
            
            box=torch.tensor(json_dict[str(key)]["found_objects"][object]["coords"])
            box_area=json_dict[str(key)]["found_objects"][object]["area"]
            masks=model.predict_sam(image_pil,box)
            masks = masks.squeeze(1)
            if len(masks)>0:
                mask_area=masks.sum()
                new_dict[key]["found_objects"][object]={"coords":box.tolist(),"box_area":box_area,"mask_area":mask_area.item()}
    with open(f"new_results{str(i)}.json","w") as f:
        r=json.dumps(new_dict)
        f.write(r)

  0%|          | 9/10000 [00:37<11:33:04,  4.16s/it]


KeyboardInterrupt: 